In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

from function_to_check_hpp import *

In [3]:
company = 'mgl'

In [4]:
df = pd.read_parquet(f"s3://mega-dev-lake/RawData/D365/Product/{company}/data.parquet")

In [5]:
itable = pd.read_parquet(
    f"s3://mega-lake/RawData/SOH/INVENTTABLE/{company}/data.parquet"
)
itable = itable.drop_duplicates(subset=["CODEBARS"], keep="last")

df_current = df[df["PullStatus"] == "Current"]
df_current = df_current.merge(
    itable[["CODEBARS", "HPP"]].rename(columns={"CODEBARS": "CodeBars"}),
    "left",
    on=["CodeBars"],
    suffixes=("", "Ax"),
).fillna({"HPPAx": 0})

df_current["HPP"] = np.where(
    df_current["HPP"] == 0, df_current["HPPAx"], df_current["HPP"]
)

df_final = pd.concat([df[df["PullStatus"] != "Current"], df_current]).fillna(
    {"HPPAx": 0}
)

In [6]:
df_mp = pd.read_parquet(f"s3://mega-dev-lake/RawData/D365/Product/mpr/data.parquet")
df_mp = df_mp[df_mp["PullStatus"] == "Current"]
df_mp = df_mp.drop_duplicates(subset=["CodeBars"])

df_current = df_final[
    (df_final["PullStatus"] == "Current")
    & (
        df_final["UmbrellaBrand"].isin(
            [
                "Minimal",
                "Mens top",
                "Menstop",
                "MOC",
                "Moc",
                "Manzone",
                "Edwin",
                "Due",
            ]
        )
    )
]

df_current = df_current.merge(
    df_mp[["CodeBars", "HPP"]],
    "left",
    on=["CodeBars"],
    suffixes=("", "_inplace"),
    indicator=True,
).fillna({"HPP_inplace": 0})

In [8]:
# HPP mp di MP naik 3%
df_current["HPP_inplace"] = df_current["HPP_inplace"] * 1.03

df_current["HPP"] = np.where(
    df_current["HPP"] == 0,
    df_current["HPP_inplace"],
    df_current["HPP"],
)

In [9]:
df_current

,ItemId,AluCode,ArticleCode,ItemName,SearchName,Style,Colour,ProductNumber,VariantNumber,AluCodeDetail,CodeBars,Size,Gramasi,Unit,UmbrellaBrand,SubBrand,Department,World,Category,SubCategory,SubCategory2,Season,MonthYear,HPPInput,AverageHPP,AvgHppFinalCosting,HPP,BasicPrice,SalePrice,Stopped,ItemGroup,ItemModelGroup,ProductType,ProductSubType,InventoryType,ProductCategory,ProductLooks,Fitting,Fitting2,ModulePriceInventory,ModulePriceSales,ModulePricePurch,ModuleInventoryUnit,ModuleSalesUnit,ModulePurchUnit,ModifiedDateTimeDetail,CreatedDateTime,ModifiedDateTime,RecId,Notes,PullStatus,PullDate,HPPAx,HPP_inplace,_merge
0,2312000017609,2312000017609,2312000017609,Due Pete In Khaki Khaki,| Pete In Khaki | Khaki,Pete In Khaki,Khaki,2312000017609 : : S : : :,0000053139,A3-Casual Bermuda Chinos,1000000048926,S,0.37,None,Due,,Men,Casual,Bottom,Bermuda,Chinos,SPRING-21,01/01/2021 00:00:00,0.0,0.0,0.0,76098.18,199900.0,0.0,No,,WA,Item,Product master,Finish Good,,,Standard,,76098.18,159920.0,0.0,PCS,PCS,PCS,2024-07-10T18:09:39Z,2023-12-31T05:46:54Z,2024-04-17T08:12:32Z,5637201729,Please Check to AX 2012,Current,2024-07-12 03:39:54,76098.18,0.0,both
1,2312000017609,2312000017609,2312000017609,Due Pete In Khaki Khaki,| Pete In Khaki | Khaki,Pete In Khaki,Khaki,2312000017609 : : M : : :,0000053140,A3-Casual Bermuda Chinos,1000000048927,M,0.38,None,Due,,Men,Casual,Bottom,Bermuda,Chinos,SPRING-21,01/01/2021 00:00:00,0.0,0.0,0.0,76098.18,199900.0,0.0,No,,WA,Item,Product master,Finish Good,,,Standard,,76098.18,159920.0,0.0,PCS,PCS,PCS,2024-07-10T18:09:39Z,2023-12-31T05:46:54Z,2024-04-17T08:12:32Z,5637201729,Please Check to AX 2012,Current,2024-07-12 03:39:54,76098.18,0.0,both
2,2312000017609,2312000017609,2312000017609,Due Pete In Khaki Khaki,| Pete In Khaki | Khaki,Pete In Khaki,Khaki,2312000017609 : : L : : :,0000053141,A3-Casual Bermuda Chinos,1000000048928,L,0.39,None,Due,,Men,Casual,Bottom,Bermuda,Chinos,SPRING-21,01/01/2021 00:00:00,0.0,0.0,0.0,76098.18,199900.0,0.0,No,,WA,Item,Product master,Finish Good,,,Standard,,76098.18,159920.0,0.0,PCS,PCS,PCS,2024-07-10T18:09:39Z,2023-12-31T05:46:55Z,2024-04-17T08:12:32Z,5637201729,Please Check to AX 2012,Current,2024-07-12 03:39:54,76098.18,0.0,both
3,2312000017609,2312000017609,2312000017609,Due Pete In Khaki Khaki,| Pete In Khaki | Khaki,Pete In Khaki,Khaki,2312000017609 : : XL : : :,0000053142,A3-Casual Bermuda Chinos,1000000048929,XL,0.40,None,Due,,Men,Casual,Bottom,Bermuda,Chinos,SPRING-21,01/01/2021 00:00:00,0.0,0.0,0.0,76098.18,199900.0,0.0,No,,WA,Item,Product master,Finish Good,,,Standard,,76098.18,159920.0,0.0,PCS,PCS,PCS,2024-07-10T18:09:39Z,2023-12-31T05:46:55Z,2024-04-17T08:12:32Z,5637201729,Please Check to AX 2012,Current,2024-07-12 03:39:54,76098.18,0.0,both
4,2312000017610,2312000017610,2312000017610,Due Pete In Grey Grey,| Pete In Grey | Grey,Pete In Grey,Grey,2312000017610 : : S : : :,0000053143,D3-Casual Bermuda Chinos,1000000048930,S,0.37,None,Due,,Men,Casual,Bottom,Bermuda,Chinos,SPRING-21,01/01/2021 00:00:00,0.0,0.0,0.0,74930.00,199900.0,0.0,No,,WA,Item,Product master,Finish Good,,,Standard,,68118.18,159920.0,0.0,PCS,PCS,PCS,2024-07-10T18:09:39Z,2023-12-31T05:46:55Z,2024-04-17T08:12:34Z,5637201730,Please Check to AX 2012,Current,2024-07-12 03:39:54,74930.00,0.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73040,2312000042431,2312000042431,2312000042431,Menstop Scottie Rpt Brown_Light,Menstop Scottie Rpt Brown_Light,Scottie Rpt,Brown_Light,2312000042431 : : S : : :,0000141017,Menstop Reguler Essential Men Bottom Bermuda C...,8990300007316,S,0.00,None,Mens top,,Men,Essential,Bottom,Bermuda Casual,,WINTER-22,10/01/2022 00:00:00,0.0,72206.0,0.0,72206.00,99900.0,0.0,No,,WA,Item,Product master,Finish Good,,Solid,Regular Fit,Standard,0.00,99900.0,0.0,PCS,PCS,PCS,2024-07-10T18:09:39Z,2024-01-09T07:27:12Z